In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [2]:
data = pd.read_csv("clean-data.csv")

In [3]:
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,2,1,3,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,335,4.34,4.35,2.75


In [4]:
data.describe()

,carat,cut,color,clarity,depth,table,price,x,y,z
count,53763.000000,53763.000000,53763.000000,53763.000000,53763.000000,53763.000000,53763.000000,53763.000000,53763.000000,53763.000000
mean,0.797460,2.554247,2.593698,3.836188,61.748781,57.457207,3930.785336,5.731405,5.733299,3.539367
std,0.473136,1.027364,1.701283,1.724832,1.419309,2.226311,3985.807738,1.118563,1.110473,0.690879
min,0.200000,0.000000,0.000000,0.000000,50.800000,43.000000,326.000000,3.730000,3.680000,2.060000
25%,0.400000,2.000000,1.000000,2.000000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,2.000000,3.000000,4.000000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,3.000000,4.000000,5.000000,62.500000,59.000000,5324.000000,6.540000,6.540000,4.030000
max,5.010000,4.000000,6.000000,7.000000,73.600000,79.000000,18823.000000,10.740000,10.540000,6.980000


In [5]:
#copy data to save back up
features = data.copy()

In [6]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53763 entries, 0 to 53762
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53763 non-null  float64
 1   cut      53763 non-null  int64  
 2   color    53763 non-null  int64  
 3   clarity  53763 non-null  int64  
 4   depth    53763 non-null  float64
 5   table    53763 non-null  float64
 6   price    53763 non-null  int64  
 7   x        53763 non-null  float64
 8   y        53763 non-null  float64
 9   z        53763 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 4.1 MB


In [7]:
features.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,2,1,3,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,335,4.34,4.35,2.75


In [8]:
#select the independent and dependent variables

X = features.drop(columns='price', axis=1)
y = features['price']

In [9]:
#train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [10]:
num_features = features[["carat", "depth","table", "price","x","y","z"]]
cat_features = features[["cut", "color", "clarity"]]

In [11]:
stan = StandardScaler()

X_train = stan.fit_transform(X_train) 
X_test = stan.fit_transform(X_test)

#pipeline numerical

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ("std_scaler", StandardScaler())
])

X_train = num_pipeline.fit_transform(num_features)

**Linear Regression**

In [12]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [13]:
from sklearn.metrics import mean_squared_error
#run the prediction
lr_pred = lr.predict(X_test)

#calculate the mean square error
lr_mse = mean_squared_error(lr_pred, y_test)

#calculate the root mean squared error
lr_rmse = np.sqrt(lr_mse)
print("The root mean squared error in Linear Regression is", lr_rmse)

The root mean squared error in Linear Regression is 1301.5801002109827


**Support Vector Machines Regressor**

In [14]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

ray = np.arange(1,15,2)

parameters = {'kernel':('linear','rbf','poly','sigmoid'), 
                'C': np.logspace(-3, 3, 5),
                'gamma': np.logspace(-3, 3, 5)}

regsvr = svm.SVR()

#fit the parameters to the gridsearch

cv_regsvr = GridSearchCV(regsvr, parameters, cv= 10)


cv_regsvr.fit(X_train, y_train)


In [ ]:
print("tuned hyperparameters: (best parameters) ", cv_regsvr.best_params_)
print("accuracy : ", cv_regsvr.best_score_)

In [ ]:
pred_svr = cv_regsvr.predict(X_test)

#calculate the RMSE
mse_svr = mean_squared_error(pred_svr, y_test)
rmse_svr = np.sqrt(mse_svr)

print("The Root mean Squared error in Support Vector Machines is: ", rmse_svr)

**Random Forest Regressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

trees = np.arange(100, 500, 50)
parameters = {}

rfr = RandomForestRegressor(n_estimators= trees, random_state=0)

rfr_cv = GridSearchCV(rfr, cv=10)

rfr_cv.fit(X_train, y_train)

In [2]:
def model_rmse(reg):
    """A function to predict"""

    prediction = reg.predict(X_test)

    #calculate the RMSE
    mean_value = mean_squared_error(prediction, y_test)
    root_mean = np.sqrt(mean_value)

    return("The Algorithm is " + str(reg) + " Root Mean Square Error is: ", root_mean)